In [ ]:
#Create the following folder structure 
$ mkdir app
$ mkdir app/static
$ mkdir app/templates


#create an __init__.py file within the app folder. This will mark the app directory as a python package

from flask import Flask

app = Flask(__name__)
from app import views



#Create the views.py file within the app folder. This will contain information about what we want to do 
#with different paths of our application

from app import app

@app.route('/')
@app.route('/index')
def index():
    return "Hello, World!"


#Finally create a run.py file in the root folder. This file will start the web development server for our application

#!flask/bin/python
from app import app
app.run(debug=True)



#Now let's add more functionality like buttons to our web application

from app import app

@app.route('/')
@app.route('/index')
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Submit'}
    return '''
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>
    <button type="button">'''+button_title['title1']+'''</button>
    <br>
    <br>
    <button type="button">'''+button_title['title2']+'''</button>
  </body>
</html>
'''

#It obviously will get too difficult to manage as the application size grows. Using templates will help us in 
#making our code lot more manageable

#Code to add in index.html
"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>
    <button type="button">{{button_title['title1']}}</button>
    <br>
    <br>
    <button type="button">{{button_title['title2']}}</button>
  </body>
</html>
"""




from flask import render_template
from app import app

@app.route('/')
@app.route('/index')
def index():
    button_title = {'title1': 'Upload File', 'title2':'Submit'}
    return render_template('index.html',
                           button_title=button_title)


#We can also use if/else statements with the help of templates

#Code to add in index.html
"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>
      {% if 1==1 %}
        <button type="button">True</button>
        <br>
        <br>
      {% else %}
        <button type="button">False</button>
        <br>
        <br>  
      {% endif %}   
    <button type="button">{{button_title['title2']}}</button>
  </body>
</html>
"""


#By default the routing is assigned for GET method

from app import app
from flask import render_template
from flask import request

@app.route('/')
@app.route('/index')
def index():
    #button_title = {'title1': 'Upload File', 'title2':'Submit'}
    #return render_template('index.html',
                           #button_title=button_title)
    return request.method



#Now go to the postman chrome extension and try a GET request and POST request

from app import app
from flask import render_template
from flask import request

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    #button_title = {'title1': 'Upload File', 'title2':'Submit'}
    #return render_template('index.html',
                           #button_title=button_title)
    return request.method

#Now we will create a form so that it can take the images as input for OCR

#First lets play with forms a bit. Change the index.html to the code below and then enter something in the
#textbox and press enter

"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form>
  		<input name="q">
  	</form>
  </body>
</html>
"""

from app import app
from flask import render_template
from flask import request


@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload File', 'title2':'Submit'}
    return render_template('index.html',
                           button_title=button_title)

#Now lets assign an action to form

"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="http://www.google.com">
  		<input name="q">
  	</form>
  </body>
</html>
"""

#Now we will change the form such that it can take the images as input. Change the index.html to below

"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="/submissionsuccess" method="POST" enctype="multipart/form-data">
  		<button type="button">
  			<input type="file" name="file1">
  			{{button_title['title1']}}
  		</button>
  		<br>
  		<br>
  		<br>
  		<button type="submit"> {{button_title['title2']}} </button>
  		
  	</form>
  </body>
</html>
"""

import os
from flask import render_template, request
from app import app



APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload File', 'title2':'Submit'}
    return render_template('index.html',
                           button_title=button_title)


@app.route('/submissionsuccess',methods=['GET','POST'])
def success():

    file1 = request.files['file1']

    return "filename is "+file1.filename


#Now we will add the OCR to our code. 


import os
from flask import render_template, request
from app import app
import subprocess



APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload File', 'title2':'Convert to text'}
    return render_template('index.html',
                           button_title=button_title)
                           


@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.png'])  
    file1 = request.files['file1']
    file1.save(destination1)
      
    print destination1
    
    
    text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
    

    #return "file1 is "+file1.filename+"file2 is "+file2.filename
    return text1



#Now we will learn about tesseract in greater detail. Lets start with orientation detection

"""

"""


import os
from flask import render_template, request
from app import app
import subprocess
import re


APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Convert to text','title3':'Get text orientation'}
    return render_template('index.html',
                           button_title=button_title)

                           

def get_orientation(linktoimage):
    orientationdata = subprocess.check_output(["tesseract", linktoimage, "stdout", "-psm", "0"], stderr=subprocess.STDOUT)
    orientpattre = re.compile('Orientation in degrees: (\d+)')
    degrees = orientpattre.findall(orientationdata)
    degrees = degrees[0]
    return degrees
                    

@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.png'])  
    file1 = request.files['file1']
    file1.save(destination1)

    if request.method == 'POST':
        if request.form['submit'] == 'Convert to text':
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            return text1
        elif request.form['submit'] == 'Get orientation':
            orientation1=get_orientation(destination1)
            return orientation1
        else:
            pass # unknown
    
    



#Box files

"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="/submissionsuccess" method="POST" enctype="multipart/form-data">
  		<button type="button">
  			<input type="file" name="file1">
  			{{button_title['title1']}}
  		</button>
  		<br>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Convert to text"> {{button_title['title2']}} </button>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Get orientation"> {{button_title['title3']}} </button>
   		<br>
  		<br>
  		<button type="submit" name="submit" value="Get all coordinates"> {{button_title['title4']}} </button>		
  	</form>
  </body>
</html>
"""



#Now let's add more functionality like buttons to our web application


import os
from flask import render_template, request
from app import app
import subprocess
import re


APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Convert to text','title3':'Get text orientation','title4':'Get all coordinates'}
    return render_template('index.html',
                           button_title=button_title)

                           

def get_orientation(linktoimage):
    orientationdata = subprocess.check_output(["tesseract", linktoimage, "stdout", "-psm", "0"], stderr=subprocess.STDOUT)
    orientpattre = re.compile('Orientation in degrees: (\d+)')
    degrees = orientpattre.findall(orientationdata)
    degrees = degrees[0]
    return degrees
                    

@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.png'])  
    file1 = request.files['file1']
    file1.save(destination1)

    if request.method == 'POST':
        if request.form['submit'] == 'Convert to text':
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            return text1
        elif request.form['submit'] == 'Get orientation':
            orientation1=get_orientation(destination1)
            return orientation1
        elif request.form['submit'] == 'Get all coordinates':
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            return box1
        else:
            pass # unknown




#This will output the coordinate data for each character in the image. Now let's see an example of how 
#this can be useful. Let's say we want to get the coordinates of a particular word in the image 

"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="/submissionsuccess" method="POST" enctype="multipart/form-data">
  		<button type="button">
  			<input type="file" name="file1">
  			{{button_title['title1']}}
  		</button>
  		<br>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Convert to text"> {{button_title['title2']}} </button>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Get orientation"> {{button_title['title3']}} </button>
   		<br>
  		<br>
  		<button type="submit" name="submit" value="Get all coordinates"> {{button_title['title4']}} </button>	
    	<br>
  		<br>
  		<input type="text" name="coortext"><button type="submit" name="submit" value="Get its coordinates"> {{button_title['title5']}} </button>
   	</form>
  </body>
</html>
"""

import os
from flask import render_template, request
from app import app
import subprocess
import re


APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Convert to text','title3':'Get text orientation','title4':'Get all coordinates','title5':'Get its coordinates'}
    return render_template('index.html',
                           button_title=button_title)

                           

def get_orientation(linktoimage):
    orientationdata = subprocess.check_output(["tesseract", linktoimage, "stdout", "-psm", "0"], stderr=subprocess.STDOUT)
    orientpattre = re.compile('Orientation in degrees: (\d+)')
    degrees = orientpattre.findall(orientationdata)
    degrees = degrees[0]
    return degrees


def findcoordinates(word,text,boxdata):
    nowhitespacetext="".join(e for e in text if e.isalnum())
    coordinates={}
    newword="".join(e for e in word if e.isalnum())
    wordindex=nowhitespacetext.find(newword)
    coordata=boxdata.splitlines()
    coordinates['start']=coordata[wordindex+1]
    coordinates['end']=coordata[wordindex+len(newword)]
    return coordinates                   

@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.png'])  
    file1 = request.files['file1']
    file1.save(destination1)

    if request.method == 'POST':
        if request.form['submit'] == 'Convert to text':
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            return text1
        elif request.form['submit'] == 'Get orientation':
            orientation1=get_orientation(destination1)
            return orientation1
        elif request.form['submit'] == 'Get all coordinates':
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            return box1
        elif request.form['submit'] == 'Get its coordinates':
            word = request.form['coortext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            PatternCoor=findcoordinates(word,text1,box1)
            return PatternCoor['start']+" "+PatternCoor['end']
        else:
            pass # unknown
    
    
    




#Now let's say we want to crop out a particular part of the image which correspond to a particular word


"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="/submissionsuccess" method="POST" enctype="multipart/form-data">
  		<button type="button">
  			<input type="file" name="file1">
  			{{button_title['title1']}}
  		</button>
  		<br>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Convert to text"> {{button_title['title2']}} </button>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Get orientation"> {{button_title['title3']}} </button>
   		<br>
  		<br>
  		<button type="submit" name="submit" value="Get all coordinates"> {{button_title['title4']}} </button>	
    	<br>
  		<br>
  		<input type="text" name="coortext"><button type="submit" name="submit" value="Get its coordinates"> {{button_title['title5']}} </button>
  		<br>
  		<br>
  		<input type="text" name="croptext"><button type="submit" name="submit" value="Crop out this text"> {{button_title['title6']}} </button>		
  	</form>
  </body>
</html>
"""


import os
from flask import render_template, request
from app import app
import subprocess
import re
from PIL import Image



APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Convert to text','title3':'Get text orientation',
    'title4':'Get all coordinates','title5':'Get its coordinates', 'title6':'Crop out this text'}
    return render_template('index.html',
                           button_title=button_title)

                           

def get_orientation(linktoimage):
    orientationdata = subprocess.check_output(["tesseract", linktoimage, "stdout", "-psm", "0"], stderr=subprocess.STDOUT)
    orientpattre = re.compile('Orientation in degrees: (\d+)')
    degrees = orientpattre.findall(orientationdata)
    degrees = degrees[0]
    return degrees


def findcoordinates(word,text,boxdata):
    nowhitespacetext="".join(e for e in text if e.isalnum())
    coordinates={}
    newword="".join(e for e in word if e.isalnum())
    wordindex=nowhitespacetext.find(newword)
    coordata=boxdata.splitlines()
    coordinates['start']=coordata[wordindex+1]
    coordinates['end']=coordata[wordindex+len(newword)]
    return coordinates                   

@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.png'])  
    file1 = request.files['file1']
    file1.save(destination1)

    if request.method == 'POST':
        if request.form['submit'] == 'Convert to text':
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            return text1
        elif request.form['submit'] == 'Get orientation':
            orientation1=get_orientation(destination1)
            return orientation1
        elif request.form['submit'] == 'Get all coordinates':
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            return box1
        elif request.form['submit'] == 'Get its coordinates':
            word = request.form['coortext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            PatternCoor=findcoordinates(word,text1,box1)
            return PatternCoor['start']+" "+PatternCoor['end']
        elif request.form['submit'] == 'Crop out this text':
            #return request.form['croptext']
            
            target=os.path.join(APP_ROOT,'images')
            word = request.form['croptext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            PatternCoor=findcoordinates(word,text1,box1)
            img = Image.open(destination1)
            height = img.size[1]
            cropdim = (int(list(PatternCoor['start'].split())[1]), height- int(list(PatternCoor['end'].split())[4]), int(list(PatternCoor['end'].split())[3]), height - int(list(PatternCoor['end'].split())[2]))
            cropdim = tuple([int(x) for x in cropdim])
            crop = img.crop(cropdim)
            cropdestination1="/".join([target,'crop1.png'])
            crop.save(cropdestination1)            
            return "Done. Please check the images folder"
            
            
        else:
            pass # unknown
    
    
    


#Now finally let's say we have want to identify checkmarks. OCR wouldn't be able to identify them. We need to 
#create special code for that.




"""
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>

  	<form action="/submissionsuccess" method="POST" enctype="multipart/form-data">
  		<button type="button">
  			<input type="file" name="file1">
  			{{button_title['title1']}}
  		</button>
  		<br>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Convert to text"> {{button_title['title2']}} </button>
  		<br>
  		<br>
  		<button type="submit" name="submit" value="Get orientation"> {{button_title['title3']}} </button>
   		<br>
  		<br>
  		<button type="submit" name="submit" value="Get all coordinates"> {{button_title['title4']}} </button>	
    	<br>
  		<br>
  		<input type="text" name="coortext"><button type="submit" name="submit" value="Get its coordinates"> {{button_title['title5']}} </button>
  		<br>
  		<br>
  		<input type="text" name="croptext"><button type="submit" name="submit" value="Crop out this text"> {{button_title['title6']}} </button>	
    	<br>
  		<br>
  		<button type="submit" name="submit" value="Detect checkmark"> {{button_title['title7']}} </button>		
  	</form>
  </body>
</html>
"""

import os
from flask import render_template, request
from app import app
import subprocess
import re
from PIL import Image



APP_ROOT= os.path.dirname(os.path.abspath(__file__))

@app.route('/')
@app.route('/index',methods=['GET','POST'])
def index():
    button_title = {'title1': 'Upload Image', 'title2':'Convert to text','title3':'Get text orientation',
    'title4':'Get all coordinates','title5':'Get its coordinates', 'title6':'Crop out this text', 'title7': 'Detect checkmark'}
    return render_template('index.html',
                           button_title=button_title)

                           

def get_orientation(linktoimage):
    orientationdata = subprocess.check_output(["tesseract", linktoimage, "stdout", "-psm", "0"], stderr=subprocess.STDOUT)
    orientpattre = re.compile('Orientation in degrees: (\d+)')
    degrees = orientpattre.findall(orientationdata)
    degrees = degrees[0]
    return degrees


def findcoordinates(word,text,boxdata):
    nowhitespacetext="".join(e for e in text if e.isalnum())
    coordinates={}
    newword="".join(e for e in word if e.isalnum())
    wordindex=nowhitespacetext.find(newword)
    coordata=boxdata.splitlines()
    coordinates['start']=coordata[wordindex+1]
    coordinates['end']=coordata[wordindex+len(newword)]
    return coordinates                   

def find_checkmark(image, leftwards, radius, distance, coordinates):
    if not coordinates:
        return 0
    if type(coordinates) == list:
        coordinates = coordinates[0]
    coorarray = coordinates.split(" ")
    img = Image.open(image)
    height = img.size[1]
    
    xstart = int(coorarray[1])
    ystart = int(coorarray[2])
    xend = int(coorarray[3])
    yend = int(coorarray[4])
    
    ycenter = int((ystart + yend)/2)
    print ycenter
    
    if leftwards:
        xcenter = xstart - distance
    else:
        xcenter = xend + distance
    #print xcenter
    cropdim = (xcenter - radius, height - ycenter - radius/1.5, xcenter + radius-10, height - ycenter + radius/1.5)
    cropdim = tuple([int(x) for x in cropdim])
    crop = img.crop(cropdim)
    #crop.save("./mycrop"+ str(ystart) + ".jpg")
    
    cropheight = crop.size[1]
    cropwidth = crop.size[0]
    
    numofpixels = 0
    numofdarkpixels = 0
    pix = crop.load()
    for x in range(cropwidth):
        for y in range(cropheight):
            color = pix[x,y]
            if sum(color) < 1020:
                numofdarkpixels += 1
            numofpixels += 1
            print color
    ratio = float(numofdarkpixels) / float(numofpixels)
    print numofdarkpixels
    print numofpixels
    crop.close()
    return ratio

@app.route('/submissionsuccess',methods=['GET','POST'])
def success():
    
    target=os.path.join(APP_ROOT,'images')
    
    if not os.path.isdir(target):
        os.mkdir(target)
    destination1="/".join([target,'file1.tif'])  
    file1 = request.files['file1']
    file1.save(destination1)

    if request.method == 'POST':
        if request.form['submit'] == 'Convert to text':
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            return text1
        elif request.form['submit'] == 'Get orientation':
            orientation1=get_orientation(destination1)
            return orientation1
        elif request.form['submit'] == 'Get all coordinates':
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            mytext = "<br />".join(box1.split("\n"))
            return mytext
        elif request.form['submit'] == 'Get its coordinates':
            word = request.form['coortext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            PatternCoor=findcoordinates(word,text1,box1)
            return PatternCoor['start']+" "+PatternCoor['end']
        elif request.form['submit'] == 'Crop out this text':
            #return request.form['croptext']
            
            target=os.path.join(APP_ROOT,'images')
            word = request.form['croptext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            PatternCoor=findcoordinates(word,text1,box1)
            img = Image.open(destination1)
            height = img.size[1]
            cropdim = (int(list(PatternCoor['start'].split())[1]), height- int(list(PatternCoor['end'].split())[4]), int(list(PatternCoor['end'].split())[3]), height - int(list(PatternCoor['end'].split())[2]))
            cropdim = tuple([int(x) for x in cropdim])
            crop = img.crop(cropdim)
            cropdestination1="/".join([target,'crop1.png'])
            crop.save(cropdestination1)            
            return "Done. Please check the images folder"
        elif request.form['submit'] == 'Detect checkmark':
            #return request.form['croptext']
            word = request.form['croptext']
            text1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4"])
            box1=subprocess.check_output(['tesseract',destination1,"stdout","-l","eng","-psm","4","makebox"])
            OptionACoor=findcoordinates('Python',text1,box1)
            OptionBCoor=findcoordinates('Java',text1,box1)
            OptionCCoor=findcoordinates('Objective',text1,box1)
            
            OptionARatio=find_checkmark(destination1,True , 16, 16,OptionACoor['start'] )
            OptionBRatio=find_checkmark(destination1,True , 16, 16,OptionBCoor['start'] )
            OptionCRatio=find_checkmark(destination1,True , 16, 16,OptionCCoor['start'] )
            
            return "Python is "+str(OptionARatio)+"<br>"+"Java is "+str(OptionBRatio)+"<br>"+"Objective C is "+str(OptionCRatio)
        else:
            pass # unknown
    
    
    

